In [1]:
import sys

sys.path.append("/home/xmh/DeepCTR-Torch")

# -*- coding: utf-8 -*-
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *
import numpy as np

In [ ]:
def load_criteo():
    # Data processing code adapted from https://github.com/facebookresearch/dlrm
    # Follow steps in https://github.com/ylongqi/dlrm/blob/master/data_utils.py to generate kaggle_processed.npz
    # Or using `./download_dataset.sh criteo` command to download the processed data.
    import os
    if os.path.exists("/home/web_server/"):
        dataset_folder = "/home/web_server/"
        datapath = dataset_folder + 'kaggle_processed.npz'
    elif os.path.exists("/home/xmh/kuiba/data/"):
        dataset_folder = "/home/xmh/kuiba/data/"
        datapath = dataset_folder + 'kaggle_processed_tiny.npz'
    else:
        raise Exception("dataset not found")
    import numpy as np

    with np.load(datapath) as data:
        X_int = data["X_int"]
        X_cat = data["X_cat"]
        y = data["y"]
        counts = data["counts"]

    raw_data = dict()

    raw_data['counts'] = counts
    print("[xmh] counts is ", counts)
    X_cat = X_cat.astype(np.int32)
    X_int = np.log(X_int + 1).astype(np.float32)
    
    int_df = pd.DataFrame(X_int,columns=['I' + str(i) for i in range(1, 14)])
    cat_df = pd.DataFrame(X_cat,columns=['C' + str(i) for i in range(1, 27)])
    target_df = pd.DataFrame(y,columns=['label'])
    df = pd.concat([int_df,cat_df,target_df], axis=1)

In [5]:
data = df
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                          for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
    linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2)

train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}



In [6]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
               task='binary',
               l2_reg_embedding=1e-5, device=device)

model.compile(optimizer="adam", loss="binary_crossentropy",
              metrics=['binary_crossentropy', 'AUC'],
              optimizer_sparse=None,
              optimizer_dense_lr=0.001,
              optimizer_sparse_lr=0.001, )

model.fit(train_model_input, train[target].values,
          batch_size=32, epochs=1, validation_split=0.01, verbose=1)

pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))


0it [00:00, ?it/s]

cpu
Train on 36305771 samples, validate on 366725 samples, 1134556 steps per epoch


1it [00:04,  4.31s/it]

Iter1


2it [00:40, 13.76s/it]

...

2201s - loss:  0.5107 - binary_crossentropy:  0.5107 - auc:  0.7179 - val_binary_crossentropy:  0.4982 - val_auc:  0.7368


2301it [35:03,  1.80it/s]

Iter2301


2301it [35:11,  1.09it/s]


KeyboardInterrupt: 